In [57]:
import sys
sys.path.append('../cace/')

import numpy as np
import torch
import torch.nn as nn
import logging
import ase

import datetime

import cace
from cace.representations import Cace
from cace.modules import CosineCutoff, MollifierCutoff, PolynomialCutoff
from cace.modules import BesselRBF, GaussianRBF, GaussianRBFCentered

from cace.models.atomistic import NeuralNetworkPotential
from cace.tasks.train import TrainingTask

torch.set_default_dtype(torch.float32)

cace.tools.setup_logger(level='INFO')
cutoff = 6

xyz_path = '../Molten-NaCl/ML_AB_NaCl_liquid_Extract.xyz'
xyz = ase.io.read(xyz_path,':')
avge0 = cace.tools.compute_average_E0s(xyz)

LLLLL =0.2
val_frac = LLLLL

print("reading data")
collection = cace.tasks.get_dataset_from_xyz(train_path=xyz_path,
                                 valid_fraction=val_frac,
                                 seed=12,
                                 cutoff=cutoff,
                                 data_key={'energy': 'energy', 'forces':'forces'}, 
                                 atomic_energies=avge0 # avg
                                 )
batch_size = 2

train_loader = cace.tasks.load_data_loader(collection=collection,
                              data_type='train',
                              batch_size=batch_size,
                              )

valid_loader = cace.tasks.load_data_loader(collection=collection,
                              data_type='valid',
                              batch_size=4,
                              )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = cace.tools.init_device(use_device)
print(f"device: {device}")


reading data
2025-06-21 20:57:18.889 INFO: Loaded 1014 training configurations from '/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/ML_AB_NaCl_liquid_Extract.xyz'
2025-06-21 20:57:18.889 INFO: Loaded 1014 training configurations from '/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/ML_AB_NaCl_liquid_Extract.xyz'
2025-06-21 20:57:18.889 INFO: Loaded 1014 training configurations from '/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/ML_AB_NaCl_liquid_Extract.xyz'
2025-06-21 20:57:18.889 INFO: Loaded 1014 training configurations from '/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/ML_AB_NaCl_liquid_Extract.xyz'
2025-06-21 20:57:18.889 INFO: Loaded 1014 training configurations from '/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/ML_AB_NaCl_liquid_Extract.xyz'
2025-06-21 20:57:18.889 INFO: Loaded 1014 training configurations from '/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-f

In [58]:
import torch
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid/stage1/Ewald_q4_MP0_loss20250513_143415_model.pth")
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP0/SOG_MP0_con_min_mae_e20250518_184801_model.pth")
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP0/SOG_MP0_con_loss20250518_184801_model.pth")

# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid/SOG_MP1_ini_loss20250516_143713_model.pth")
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP1/SOG_MP1_con_loss20250518_163832_model.pth")
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP1/SOG_MP1_con_min_mae_e20250518_163832_model.pth")

# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP0/SOG_MP0_con_min_mae_e20250518_213052_model.pth")
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid/Ewald_q4_MP1_loss20250518_103452_model.pth")

# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP1/SOG_MP1_con_min_mae_e20250518_233959_model.pth")
# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP1/SOG_MP1_con_loss20250518_233959_model.pth")

# cace_nnp = torch.load("/dssg/home/acct-matxzl/matxzl/Yajie/MDNN/cace-lr-fit-main_dl/Les-NaCl/loss_data_liquid_con_MP1/SOG_MP1_con_min_mae_f20250519_165810_model.pth")
cace_nnp = torch.load("../Molten-NaCl/loss_data/SOG_MP1_con_loss20250621_204649_model.pth")
cace_nnp.to(device)

CombinePotential(
  (models): ModuleList(
    (0): NeuralNetworkPotential(
      (postprocessors): ModuleList()
      (representation): Cace(
        (node_onehot): NodeEncoder(num_classes=2)
        (node_embedding_sender): NodeEmbedding(num_classes=2, embedding_dim=3)
        (node_embedding_receiver): NodeEmbedding(num_classes=2, embedding_dim=3)
        (edge_coding): EdgeEncoder(directed=True)
        (radial_basis): BesselRBF(cutoff=6.0, n_rbf=6, trainable=True)
        (cutoff_fn): PolynomialCutoff(p=6.0, cutoff=6.0)
        (angular_basis): AngularComponent(l_max=3)
        (radial_transform): SharedRadialLinearTransform(
          (weights): ParameterList(
              (0): Parameter containing: [torch.float32 of size 6x12x9 (cuda:0)]
              (1): Parameter containing: [torch.float32 of size 6x12x9 (cuda:0)]
              (2): Parameter containing: [torch.float32 of size 6x12x9 (cuda:0)]
              (3): Parameter containing: [torch.float32 of size 6x12x9 (cuda:0)]
  

In [59]:
# nn_charges = []
# nn_charges_total = []
nn_forces = []
nn_energes = []
ref_forces = []
ref_energes = []

i=0
for batch in valid_loader:
    i= i+1
    batch.cuda()
    out = cace_nnp(batch)
    # nn_q = out["q"]
    # nn_tq = out["tot_q"]
    nn_f = out["CACE_forces"]
    nn_e = out["CACE_energy"]

    ref_f = batch.forces
    ref_e = batch.energy
    # nn_charges.append(nn_q.squeeze().cpu().detach().numpy())
    # nn_charges_total.append(nn_tq.squeeze().cpu().detach().numpy())
    nn_forces.append(nn_f.squeeze().cpu().detach().numpy())
    nn_energes.append(nn_e.squeeze().cpu().detach().numpy())
    ref_forces.append(ref_f.squeeze().cpu().detach().numpy())
    ref_energes.append(ref_e.squeeze().cpu().detach().numpy())
    
# nn_charges = np.hstack(nn_charges)
# nn_charges_total = np.hstack(nn_charges_total)
nn_forces = np.vstack(nn_forces)
nn_energes = np.hstack(nn_energes)
ref_forces = np.vstack(ref_forces)
ref_energes = np.hstack(ref_energes)

In [60]:
def force_rmspe_squared_ratio(pred_forces, ref_forces):
    pred_forces = np.asarray(pred_forces).reshape(-1, 3)
    ref_forces = np.asarray(ref_forces).reshape(-1, 3)

    diff_sq = np.sum((pred_forces - ref_forces)**2, axis=1)           # ||ΔF||^2
    ref_sq = np.sum(ref_forces**2, axis=1)                             # ||F||^2
    # print(diff_sq.shape,ref_sq.shape,np.mean(diff_sq),np.mean(ref_sq))
    rmsd1 = np.sqrt(np.mean(diff_sq))  # RMSD
    rmsd2 = np.sqrt(np.mean(ref_sq))   #np.std(ref_forces) 
    rmspe = (rmsd1 / rmsd2) * 100
    return rmspe

def enegry_rmspe_squared_ratio(pred_forces, ref_forces):
    # pred_forces = np.asarray(pred_forces).reshape(-1, 1)
    # ref_forces = np.asarray(ref_forces).reshape(-1, 1)

    diff_sq = (pred_forces - ref_forces)**2         # ||ΔF||^2
    ref_sq = ref_forces**2                      # ||F||^2
    # print(diff_sq.shape,ref_sq.shape,np.mean(diff_sq),np.mean(ref_sq),np.std(pred_forces-ref_forces),np.std(ref_forces))   
    rmsd1 = np.sqrt(np.mean(diff_sq))  # RMSD
    rmsd2 = np.sqrt(np.mean(ref_sq)) # np.std(ref_forces) 
    rmspe = (rmsd1 / rmsd2) * 100
    return rmspe

print("Force:",force_rmspe_squared_ratio(nn_forces,ref_forces))
print("Energy:",enegry_rmspe_squared_ratio(nn_energes,ref_energes))

Force: 1.9313473254442215
Energy: 1.8868720158934593
